In [1]:
import basedosdados as bd
import geopandas as gpd
from shapely import wkt

In [2]:
# dados de Nascidos Vivos

query = """
SELECT ano, id_municipio_nascimento, data_nascimento, hora_nascimento, sexo, 
       peso, apgar1, apgar5, idade_mae, tipo_parto
FROM basedosdados.br_ms_sinasc.microdados
WHERE (ano BETWEEN 1998 AND 2020) AND (sigla_uf = 'RJ')
"""

df = bd.read_sql(query, billing_project_id="nascidosvivos")

Downloading: 100%|██████████| 5137387/5137387 [08:49<00:00, 9710.56rows/s] 


In [3]:
df.to_csv('df.csv', index=True)

In [4]:
# dados das cidades do estado do Rio de Janeiro

query = """
SELECT DISTINCT M.id_municipio, SC.nome_municipio, M.geometria
FROM basedosdados.br_geobr_mapas.municipio M
INNER JOIN basedosdados.br_geobr_mapas.setor_censitario_2010 SC
ON M.id_municipio = SC.id_municipio
WHERE SC.sigla_uf = 'RJ'
"""

cidades = bd.read_sql(query, billing_project_id="nascidosvivos")

Downloading: 100%|██████████| 92/92 [00:01<00:00, 64.71rows/s]


In [5]:
cidades['geometria'] = cidades['geometria'].apply(wkt.loads)
cidades = gpd.GeoDataFrame(cidades, geometry='geometria')

cidades['id_municipio'] = cidades['id_municipio'].astype(int)
cidades['id_municipio'] = cidades['id_municipio'].astype(str)

cidades.to_file('../Nascidos-Vivos-Viz/cidades.geojson', driver='GeoJSON')
cidades.to_csv('cidades.csv', index=True)